
# Example of High Gamma Filter

Below is a code sample for extracting high gamma power from a raw data file, followed by permutation cluster stats on that high gamma power data


### working version 12/1/23

### try gregs suggestion of using make_data_same to destroy the fixation cross

use window stats with perm testing (0 to 0.5, 0.5 to 1, 0 to 1 sec relative to stim onset)

In [ ]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans, channel_names_to_indices, \
    filter_and_average_epochs, permutation_test, perform_permutation_test_across_electrodes, \
    perform_permutation_test_within_electrodes, add_accuracy_to_epochs, save_sig_chans_with_reject

import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests

# Directory where your .npy files are saved
npy_directory = r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\accArrays'  # Replace with your directory path

# Dictionary to hold the data
acc_array = {}

# Iterate over each file in the directory
for file in os.listdir(npy_directory):
    if file.endswith('.npy'):
        # Construct the full file path
        file_path = os.path.join(npy_directory, file)
        # Load the numpy array from the file
        acc_array[file.split('_')[0]] = np.load(file_path)

# Now you have a dictionary where each key is the subject ID
# and the value is the numpy array of accuracies for that subject.
        
combined_data = pd.read_csv(r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\combinedData.csv')

# subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
subjects = ['D0069']

In [ ]:
### after loading in HG_ev1_rescaled, then grab only the correct trials. Think I need to load in combined_data here too?
### then load in the perm testing functions (put these into misc functions now that they work, and just load them into roi analysis and this too)
###


time_windows = {
    "Stimulus_fixationCrossBase_0.2sec_window_0to0.5": (2048,3072), #do these based on the sample rate, so need to load in filt before looping over windows. Kind of annoying honestly.
    "Stimulus_fixationCrossBase_0.2sec_window_0.5to1": (3072,4096),
    "Stimulus_fixationCrossBase_0.2sec_window_0to1": (2048,4096)
}

for window in time_windows:
    for sub in subjects:
        start_idx, end_idx = int(time_windows[window][0]), int(time_windows[window][1]) # i think these need to be ints 2/27
        task = 'GlobalLocal'
        output_name = window #prob turn this into a function where i can set the window length as an input and it'll get added to the output name
        events = ["Stimulus"]
        times = (-1,1.5)
        base_times = [-0.2,0]
        LAB_root = None
        channels = None
        full_trial_base = False

        if LAB_root is None:
            HOME = os.path.expanduser("~")
            if os.name == 'nt':  # windows
                LAB_root = os.path.join(HOME, "Box", "CoganLab")
            else:  # mac
                LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                        "CoganLab")

        layout = get_data(task, root=LAB_root)
        filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                            extension='.edf', desc='clean', preload=False)
        # Access the sampling frequency
        sampling_frequency = filt.info['sfreq']
    
        # Print the sampling frequency
        print(f"Subject {sub} has a sampling frequency of {sampling_frequency} Hz.")
        
        save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        good = crop_empty_data(filt)
        # %%

        print(f"good channels before dropping bads: {len(good.ch_names)}")
        print(f"filt channels before dropping bads: {len(filt.ch_names)}")

        good.info['bads'] = channel_outlier_marker(good, 3, 2)
        print("Bad channels in 'good':", good.info['bads'])

        filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
        good.drop_channels(good.info['bads'])

        print("Bad channels in 'good' after dropping once:", good.info['bads'])

        print(f"good channels after dropping bads: {len(good.ch_names)}")
        print(f"filt channels after dropping bads: {len(filt.ch_names)}")

        good.load_data()

        # If channels is None, use all channels
        if channels is None:
            channels = good.ch_names
        else:
            # Validate the provided channels
            invalid_channels = [ch for ch in channels if ch not in good.ch_names]
            if invalid_channels:
                raise ValueError(
                    f"The following channels are not valid: {invalid_channels}")

            # Use only the specified channels
            good.pick_channels(channels)

        ch_type = filt.get_channel_types(only_data_chs=True)[0]
        good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

        # Create a baseline EpochsTFR using the stimulus event

        adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
        trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
        outliers_to_nan(trials, outliers=10)
        HG_base = gamma.extract(trials, copy=False, n_jobs=1)
        crop_pad(HG_base, "0.5s")

        all_epochs_list = []

        for event in events:
        # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
            times_adj = [times[0] - 0.5, times[1] + 0.5]
            trials = trial_ieeg(good, event, times_adj, preload=True,
                                reject_by_annotation=False)
            all_epochs_list.append(trials)

        # Concatenate all trials
        all_trials = mne.concatenate_epochs(all_epochs_list)

        outliers_to_nan(all_trials, outliers=10)
        HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
        print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
        crop_pad(HG_ev1, "0.5s")
        print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

        ###
        print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
        print(f"Shape of HG_base._data: {HG_base._data.shape}")

        # sig1 = HG_ev1._data
        # sig2 = HG_base._data
        # sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
        # sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
        # sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
        # print(f"Shape of sig1: {sig1.shape}")
        # print(f"Shape of sig2: {sig2.shape}")
        # print(f"Shape of sig3: {sig3.shape}")
        # print(f"Shape of sig4: {sig4.shape}")
        # print(f"Shape of sig5: {sig5.shape}")

        # sig2 = sig5
        # # Directly reassign the modified signal data to the HG_base._data attribute (THIS MAY BREAK THINGS BUT ITS CORRECT 2/13)
        # HG_base._data = sig2

        ### filter for accurate trials in HG ev1 
        # Build the filtering condition
        sub_without_zeroes = "D" + sub[1:].lstrip('0') 
        sub_behavioral_rows = (combined_data['subject_ID'] == sub_without_zeroes) # this indexes using the subject without zeroes in the name. Confusing. I know.

        # Filter combinedData for the specific subject and conditions
        sub_behavioral_data = combined_data[sub_behavioral_rows]

        if sub in acc_array:
            trial_counts = sub_behavioral_data['trialCount'].values.astype(int)
            accuracy_data = [acc_array[sub][i-1] for i in trial_counts if i-1 < len(acc_array[sub])] # Subtract 1 here for zero-based indexing in acc array.
            
            # Now pass trial_counts along with accuracy_data to raw HG_ev1
            HG_ev1 = add_accuracy_to_epochs(HG_ev1, accuracy_data)
            
        # Separate accurate and all trials data
        accurate_HG_ev1_data = HG_ev1[HG_ev1.metadata['accuracy'] == 1.0].get_data()
        all_HG_ev1_data = HG_ev1.get_data().copy()

        # Mark inaccurate trials as NaNs in the all_epochs_data
        inaccurate_indices = HG_ev1.metadata['accuracy'] != 1.0
        all_HG_ev1_data[inaccurate_indices, :, :] = np.nan
        
        # do stats stuff here. 2/22 I think we need to do this before we decimate.
        # Calculate time average within the specified window
        time_avg_signal = np.nanmean(all_HG_ev1_data[:, :, start_idx:end_idx], axis=2) #average across specified time window for data
        time_avg_base = np.nanmean(HG_base.get_data().copy()[:,:,:], axis=2) #average across all time for baseline. Use a copy of the baseline so we don't mess it up.
        p_values = perform_permutation_test_within_electrodes([time_avg_signal], [time_avg_base], one_tailed=True) #update these functions to not need list inputs later.
        reject, p_values_adjusted = multipletests(p_values, alpha=0.05, method='fdr_bh')[:2] # reject is a boolean array of whether each channel passed significance threshold after adjusting for multiple comparisons. Adjusted_p_values are the actual adjusted p values.
        # check p_values_adjusted after lab meeting! 2/13

        HG_base.decimate(2)
        HG_ev1.decimate(2)

        HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

        HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
        HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

        HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
        HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

        HG_ev1_evoke_stderr = HG_ev1.standard_error()
        HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

 
        # Determine which channels are significant based on the reject array
        channels = good.ch_names
        save_sig_chans_with_reject(output_name, reject, channels, sub, save_dir)

        #save all channels with their indices 
        save_channels_to_file(channels, sub, task, save_dir)

        # Save HG_ev1
        HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

        # Save HG_base
        HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

        # Save HG_ev1_rescaled
        HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

        # Save HG_ev1_evoke
        HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

        # Save HG_ev1_evoke_rescaled
        HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

use time point cluster stats for determining stimulus significance (old method as of 2/13/24)

updated this one 2/29, once it's tested and works, then turn into a function and delete other cells below

uncomment things and delete the subjects variable once we get the mat shape 3/10

In [ ]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5),
                      base_times=(-0.5, 0), LAB_root=None, channels=None,
                      full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.

    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task.
    High gamma (HG) will be computed, and statistics will be calculated and plotted.
    The results will be saved to output files.
    """
    pass

for sub in subjects:

    task = 'GlobalLocal'
    output_name = "Stimulus_fixationCrossBase_1sec_mirror_0to1Test"
    events = ["Stimulus"]
    times = (0,1)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)



    sig_chans_filename = os.path.join(save_dir, f'sig_chans_{sub}_{output_name}.json')
    sig_chans = load_sig_chans(sig_chans_filename)

    # Assuming `mat` is your array and `save_dir` is the directory where you want to save it
    mat_save_path = os.path.join(save_dir, f'{output_name}_mat.npy')

    # Save the mat array
    np.save(mat_save_path, mat)

### loop through everyone and do congruency 

In [ ]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5),
                      base_times=(-0.5, 0), LAB_root=None, channels=None,
                      full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.

    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task.
    High gamma (HG) will be computed, and statistics will be calculated and plotted.
    The results will be saved to output files.
    """
    pass

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_s25and75_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/i75/s25", "Stimulus/c75/s25", "Stimulus/i25/s25", "Stimulus/c25/s25", "Stimulus/i75/s75", "Stimulus/c75/s75", "Stimulus/i25/s75", "Stimulus/c25/s75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)


for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_r25and75_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/i75/r25", "Stimulus/c75/r25", "Stimulus/i25/r25", "Stimulus/c25/r25", "Stimulus/i75/r75", "Stimulus/c75/r75", "Stimulus/i25/r75", "Stimulus/c25/r75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)



In [ ]:
from IPython.display import clear_output
clear_output(wait=True)

In [ ]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5),
                      base_times=(-0.5, 0), LAB_root=None, channels=None,
                      full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.

    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task.
    High gamma (HG) will be computed, and statistics will be calculated and plotted.
    The results will be saved to output files.
    """
    pass

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_c75_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/c75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_i75_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/i75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)



### do interaction effects (is, ir, cs, cr)

In [ ]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5),
                      base_times=(-0.5, 0), LAB_root=None, channels=None,
                      full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.

    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task.
    High gamma (HG) will be computed, and statistics will be calculated and plotted.
    The results will be saved to output files.
    """
    pass

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_is_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/i25/s25", "Stimulus/i25/s75", "Stimulus/i75/s25", "Stimulus/i75/s75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)


for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_ir_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/i25/r25", "Stimulus/i25/r75", "Stimulus/i75/r25", "Stimulus/i75/r75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)



In [ ]:
from IPython.display import clear_output
clear_output(wait=True)

In [ ]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5),
                      base_times=(-0.5, 0), LAB_root=None, channels=None,
                      full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.

    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task.
    High gamma (HG) will be computed, and statistics will be calculated and plotted.
    The results will be saved to output files.
    """
    pass

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_cs_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/c25/s25", "Stimulus/c25/s75", "Stimulus/c75/s25", "Stimulus/c75/s75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)


for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_cr_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/c25/r25", "Stimulus/c25/r75", "Stimulus/c75/r25", "Stimulus/c75/r75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)

do switch type proportions by block

In [ ]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5),
                      base_times=(-0.5, 0), LAB_root=None, channels=None,
                      full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.

    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task.
    High gamma (HG) will be computed, and statistics will be calculated and plotted.
    The results will be saved to output files.
    """
    pass

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_s75_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/i25/s75", "Stimulus/i75/s75", "Stimulus/c25/s75", "Stimulus/c75/s75"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_r75_fixationCrossBase_1sec_mirror"
    events = ["Stimulus/i25/r75", "Stimulus/i75/r75", "Stimulus/c25/r75", "Stimulus/c75/r75"]    
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Create a baseline EpochsTFR using the stimulus event

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    all_epochs_list = []

    for event in events:
    # Epoching and HG extraction for each specified event. Then concatenate all trials epochs objects together (do Stimulus/c25 and Stimulus/c75 for example, and combine to get all congruent trials)
        times_adj = [times[0] - 0.5, times[1] + 0.5]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)
        all_epochs_list.append(trials)

    # Concatenate all trials
    all_trials = mne.concatenate_epochs(all_epochs_list)

    outliers_to_nan(all_trials, outliers=10)
    HG_ev1 = gamma.extract(all_trials, copy=True, n_jobs=1)
    print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
    crop_pad(HG_ev1, "0.5s")
    print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

    HG_base.decimate(2)
    HG_ev1.decimate(2)

    HG_ev1_avgOverTime = np.nanmean(HG_ev1.get_data(), axis=2)
    HG_ev1_rescaled_avgOverTime = np.nanmean(HG_ev1_rescaled.get_data(), axis=2)

    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    # if event == "Stimulus":
    #     print('plotting stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1)) #this line is not finishing...
    #     print('plotted')
    #     # for ax in fig.axes:
    #     #     ax.axvline(x=avg_RT, color='r', linestyle='--')
    #     print('about to save')
    #     fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
    #     print('saved')
    # else:
    #     print('about to plot if not stimulus')
    #     fig = HG_ev1_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
    #     print('plotted non stimulus')
    #     fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

    # Save HG_ev1
    HG_ev1.save(f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif', overwrite=True)

    # Save HG_base
    HG_base.save(f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif', overwrite=True)

    # Save HG_ev1_rescaled
    HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif', overwrite=True)

    # Save HG_ev1_evoke
    HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke-epo.fif', overwrite=True)

    # Save HG_ev1_evoke_rescaled
    HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

    ###
    print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
    print(f"Shape of HG_base._data: {HG_base._data.shape}")

    sig1 = HG_ev1._data
    sig2 = HG_base._data
    sig3 = make_data_same(sig2, (sig2.shape[0],sig2.shape[1],sig2.shape[2]+1)) # originally we want to make the baseline the same shape as the signal. We still want to do that, but first, we'll make it bigger to reflect it once, then back to normal to randomly offset it and remove fixation cross effects.
    sig4 = make_data_same(sig3, sig2.shape) #here we do the random offset, we know that sig3 is bigger than sig1 by 1 in the time dimension so it will get randomly sliced.
    sig5 = make_data_same(sig4, sig1.shape) #and now sig4 should be sig2 but with a random offset, and we can then set it equal to sig1's shape like the original plan.
    print(f"Shape of sig1: {sig1.shape}")
    print(f"Shape of sig2: {sig2.shape}")
    print(f"Shape of sig3: {sig3.shape}")
    print(f"Shape of sig4: {sig4.shape}")
    print(f"Shape of sig5: {sig5.shape}")

    sig2 = sig5

    mat = time_perm_cluster(sig1, sig2, 0.05, n_jobs=6, ignore_adjacency=1)
    fig = plt.figure()
    plt.imshow(mat, aspect='auto')
    fig.savefig(save_dir + f'_{output_name}_stats.png', dpi=300)

    channels = good.ch_names

    #save channels with their indices 
    save_channels_to_file(channels, sub, task, save_dir)

    # save significant channels to a json
    save_sig_chans(f'{output_name}', mat, channels, sub, save_dir)


    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
    sig_chans_filename = f'{base_path}\\{sub}\\sig_chans_{sub}_{output_name}.json'
    sig_chans = load_sig_chans(sig_chans_filename)



### plot each significant channel with its trace and timepoints of significance

first, load in previously generated hg ev1 and hg base for stimulus significance from baseline

turn this into a loop over all three time windows and all 12 subjects

In [ ]:
from PIL import Image, ImageChops

def trim_whitespace(image):
    """
    Trims the whitespace from an image.
    """
    bg = Image.new(image.mode, image.size, image.getpixel((0, 0)))
    diff = ImageChops.difference(image, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return image.crop(bbox)
    return image  # If no change

def plot_channels_on_grid_windows(evoke_data, std_err_data, channels_subset, time_windows, sig_chans, sample_rate, plot_x_dim=6, plot_y_dim=6):
    """
    Plots evoked EEG/MEG data for a subset of channels on a grid, overlaying significance markers for specified time windows.

    Parameters:
    - evoke_data: mne.Evoked object
        The evoked data to be plotted. This object contains the averaged EEG/MEG data over epochs.
    - std_err_data: 
        The standard error of the evoked data to be plotted
    - channels_subset: list of str
        A list of channel names to be plotted. Each channel name must correspond to a channel in `evoke_data`.
    - time_windows: dict
        A dictionary where keys are strings representing the names of the time windows of interest, and values are tuples
        indicating the start and end indices (in samples) of these windows.
    - sig_chans: dict
        A dictionary where keys are the names of the time windows (matching those in `time_windows`) and values are lists
        of channel names (str) that are significant within those windows.
    - sample_rate: float
        The sampling rate of the data, in Hz. Used to convert sample indices in `time_windows` to time in seconds.
    - plot_x_dim: int, optional (default=6)
        The number of columns in the grid layout for plotting the channels.
    - plot_y_dim: int, optional (default=6)
        The number of rows in the grid layout for plotting the channels.

    Returns:
    - fig: matplotlib.figure.Figure object
        The figure object containing the grid of plots. Each plot shows the evoked data for a channel, with significance
        markers overlaid for the specified time windows.
    """
    fig, axes = plt.subplots(plot_y_dim, plot_x_dim, figsize=(20, 12))  # Adjusted to match your desired layout
    fig.suptitle("Channels with Significance Overlay for Different Time Windows")
    axes_flat = axes.flatten()

    # Define colors for each time window
    colors = ['red', 'green', 'blue']
    window_names = list(time_windows.keys())

    for channel, ax in zip(channels_subset, axes_flat):
        stderr = stderr_data.data[channel_to_index[channel], :]
        # Plot the channel data with times in seconds
        ax.plot(evoke_data.times, evoke_data.data[channel_to_index[channel], :])
         # Add the standard error shading
        ax.fill_between(evoke_data.times, evoke_data.data[channel_to_index[channel], :] - stderr, evoke_data.data[channel_to_index[channel], :] + stderr, alpha=0.2)

        max_y_value = np.max(evoke_data.data[channel_to_index[channel], :])  # Find max y-value for significance lines
        # Assuming the epochs start 1 second before the stimulus/event
        epoch_start_time = -1  # Start time of epochs in seconds

        for window_index, window_name in enumerate(window_names):
            if channel in sig_chans[window_name]:
                start_idx, end_idx = time_windows[window_name]
                # Convert sample indices to times in seconds
                start_time = (start_idx / sample_rate) + epoch_start_time
                end_time = (end_idx / sample_rate) + epoch_start_time
                # Determine y-position for the significance line, adjusting to avoid overlap
                y_position = max_y_value - (window_index * 0.02 * max_y_value)  # Adjust overlap offset here

                # Cycle through colors for each time window
                color = colors[window_index % len(colors)]
                ax.hlines(y=y_position, xmin=start_time, xmax=end_time, color=color, linewidth=2, label=f"{window_name}: {color}")

        ax.set_title(channel)

    # Create a legend for the first subplot (if desired) to explain the colors
    if len(axes_flat) > 0 and len(window_names) > 0:
        handles, labels = axes_flat[0].get_legend_handles_labels()
        fig.legend(handles, labels, loc='upper right', title="Time Windows & Colors")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make space for the legend
    return fig

sig_chans = {}

for sub in subjects:
    task = 'GlobalLocal'
    LAB_root = None
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    sample_rate = filt.info['sfreq'] # get sampling rate, should be 2048 Hz
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    time_windows = {
        "Stimulus_fixationCrossBase_0.2sec_window_0to0.5": (sample_rate,1.5*sample_rate), #actually grab from 1 to 1.5 because the epochs start at -1 second before stim onset
        "Stimulus_fixationCrossBase_0.2sec_window_0.5to1": (1.5*sample_rate,2*sample_rate),
        "Stimulus_fixationCrossBase_0.2sec_window_0to1": (sample_rate,2*sample_rate)
    }

    for window in time_windows:
        output_name = window

        # Define file paths
        HG_ev1_file = f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif'
        HG_base_file = f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif'
        HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'

        # Load the epochs and evoked objects
        HG_ev1 = mne.read_epochs(HG_ev1_file)
        HG_base = mne.read_epochs(HG_base_file)
        HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
        HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
        HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))
        HG_ev1_evoke_stderr = HG_ev1.standard_error()
        HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

        channels = [] # load in all channels
        channel_to_index = {}
        channel_file = os.path.join(save_dir, f'channels_{sub}_GlobalLocal.txt') 
        with open(channel_file, 'r') as f:
            for line in f:
                index, channel = line.strip().split(': ')
                channels.append(channel)
                channel_to_index[channel] = int(index)

        sig_chans_filename = os.path.join(save_dir, f'sig_chans_{sub}_{output_name}.json') # load in sig channels
        sig_chans[window] = load_sig_chans(sig_chans_filename)

    # now plot 6x6 grid of 36 channels on one plot, for the z-scored signal
    plot_x_dim = 6
    plot_y_dim = 6
    channels_per_fig = plot_x_dim * plot_y_dim

    # Iterate over all channels in chunks and plot them with z-scored signal
    for i in range(0, len(channels), channels_per_fig):
        channels_subset = channels[i:i+channels_per_fig]
        fig = plot_channels_on_grid_windows(HG_ev1_evoke_rescaled, HG_ev1_evoke_rescaled_stderr, channels_subset, time_windows, sig_chans, sample_rate, plot_x_dim, plot_y_dim)
        combined_plot_path_rescaled = os.path.join(save_dir, f'{sub}_zscore_{output_name}_combinedChannelTracesAndWindowsSignificance_Page_{i//channels_per_fig + 1}.png')
        fig.savefig(combined_plot_path_rescaled)
        plt.close(fig)

    for i in range(0, len(channels), channels_per_fig):
        channels_subset = channels[i:i+channels_per_fig]
        fig = plot_channels_on_grid_windows(HG_ev1_evoke, HG_ev1_evoke_stderr, channels_subset, time_windows, sig_chans, sample_rate, plot_x_dim, plot_y_dim)
        combined_plot_path_rescaled = os.path.join(save_dir, f'{sub}_raw_{output_name}_combinedChannelTracesAndWindowsSignificance_Page_{i//channels_per_fig + 1}.png')
        fig.savefig(combined_plot_path_rescaled)
        plt.close(fig)

this below code is for when using the time perm cluster stats to determine significance timepoint by timepoint

In [ ]:
# UNTESTED 2/29, NEED TO RERUN STATS TO SAVE MAT FIRST.
def plot_channels_on_grid_time_perm_cluster(evoke_data, std_err_data, channels_subset, mat, sample_rate=2048, plot_x_dim=6, plot_y_dim=6):
    """
    Plots evoked EEG/MEG data for a subset of channels on a grid, overlaying significance markers for specified time windows.

    Parameters:
    - evoke_data: mne.Evoked object
        The evoked data to be plotted. This object contains the averaged EEG/MEG data over epochs.
    - std_err_data: 
        The standard error of the evoked data to be plotted
    - channels_subset: list of str
        A list of channel names to be plotted. Each channel name must correspond to a channel in `evoke_data`.
    - mat: numpy.array
        A binary matrix (same shape as evoke_data) indicating significant data points (1 for significant, 0 for non-significant).
    - sample_rate: float
    - sample_rate: float
        The sampling rate of the data, in Hz. Used to convert sample indices in `time_windows` to time in seconds.
    - plot_x_dim: int, optional (default=6)
        The number of columns in the grid layout for plotting the channels.
    - plot_y_dim: int, optional (default=6)
        The number of rows in the grid layout for plotting the channels.

    Returns:
    - fig: matplotlib.figure.Figure object
        The figure object containing the grid of plots. Each plot shows the evoked data for a channel, with significance
        markers overlaid for the specified time windows.
    """
    fig, axes = plt.subplots(plot_x_dim, plot_y_dim, figsize=(20, 12))
    fig.suptitle("Channels with Significance Overlay")
    axes_flat = axes.flatten()

    for channel, ax in zip(channels_subset, axes_flat):
        stderr = std_err_data.data[channel_to_index[channel], :]
        time_in_seconds = np.arange(0, len(mat[channel_to_index[channel]])) / sample_rate  # Should be 2048 Hz sample rate
        sig_data_in_seconds = np.array(mat[channel_to_index[channel]])
        ax.plot(evoke_data.times, evoke_data.data[channel_to_index[channel], :])
         # Add the standard error shading
        ax.fill_between(evoke_data.times, evoke_data.data[channel_to_index[channel], :] - stderr, evoke_data.data[channel_to_index[channel], :] + stderr, alpha=0.2)

        # Find the maximum y-value for the current channel
        max_y_value = np.max(evoke_data.data[channel_to_index[channel], :])

        # Overlay significance as a horizontal line at the max y-value
        significant_points = np.where(sig_data_in_seconds == 1)[0]
        for point in significant_points:
            ax.hlines(y=max_y_value, xmin=time_in_seconds[point]-1, xmax=time_in_seconds[point] + 0.005 - 1, color='red', linewidth=1) # subtract 1 cuz the sig time is from 0 to 2.5, while the high gamma time is from -1 to 1.5

        ax.set_title(channel)

    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    return fig

plot_x_dim = 6
plot_y_dim = 6
channels_per_fig = plot_x_dim * plot_y_dim

sig_chans = {}

for sub in subjects:
    task = 'GlobalLocal'
    LAB_root = None
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    sample_rate = filt.info['sfreq'] # get sampling rate, should be 2048 Hz
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    output_name = 'Stimulus_fixationCrossBase_1sec_mirror_0to1test'

    # Define file paths
    HG_ev1_file = f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif'
    HG_base_file = f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif'
    HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'

    # Load the epochs and evoked objects
    HG_ev1 = mne.read_epochs(HG_ev1_file)
    HG_base = mne.read_epochs(HG_base_file)
    HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))
    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    mat_save_path = os.path.join(save_dir, f'{output_name}_mat.npy')
    mat = np.load(mat_save_path)

    channels = [] # load in all channels
    channel_to_index = {}
    channel_file = os.path.join(save_dir, f'channels_{sub}_GlobalLocal.txt') 
    with open(channel_file, 'r') as f:
        for line in f:
            index, channel = line.strip().split(': ')
            channels.append(channel)
            channel_to_index[channel] = int(index)
    
    # Iterate over all channels in chunks of channels_per_fig (plot_x_dim * plot_y_dim) and plot them
    for i in range(0, len(channels), channels_per_fig):
        channels_subset = channels[i:i+channels_per_fig]
        fig = plot_channels_on_grid_time_perm_cluster(HG_ev1_evoke_rescaled, HG_ev1_evoke_rescaled_stderr, channels_subset, mat, plot_x_dim, plot_y_dim, sample_rate=sample_rate)
        combined_plot_path = os.path.join(save_dir, f'{sub}_zscore_{output_name}_combinedChannelTracesAndTimePermClusterSignificance_Page_{i//channels_per_fig + 1}.png')
        fig.savefig(combined_plot_path)
        plt.close(fig)

        # Iterate over all channels in chunks of channels_per_fig (plot_x_dim * plot_y_dim) and plot them
    for i in range(0, len(channels), channels_per_fig):
        channels_subset = channels[i:i+channels_per_fig]
        fig = plot_channels_on_grid_time_perm_cluster(HG_ev1_evoke, HG_ev1_evoke_stderr, channels_subset, mat, plot_x_dim, plot_y_dim, sample_rate=sample_rate)
        combined_plot_path = os.path.join(save_dir, f'{sub}_raw_{output_name}_combinedChannelTracesAndTimePermClusterSignificance_Page_{i//channels_per_fig + 1}.png')
        fig.savefig(combined_plot_path)
        plt.close(fig)

### z-scored signal

In [ ]:
# for raw traces, just plot HG_ev1_evoke instead of HG_ev1_evoke_rescaled. And for the standard error, use the HG_ev1_evoke_stderr instead of HG_ev1_evoke_rescaled_stderr

### raw traces

In [ ]:
# Assuming all imports and previous definitions are in place

def plot_channels_on_grid_time_perm_cluster_raw(channels_subset):
    fig, axes = plt.subplots(6, 10, figsize=(20, 33))
    fig.suptitle("Channels with Significance Overlay")
    axes_flat = axes.flatten()

    for channel, ax in zip(channels_subset, axes_flat):
        stderr = HG_ev1_evoke_stderr.data[channel_to_index[channel], :]
        time_in_seconds = np.arange(0, len(mat[channel_to_index[channel]])) / sample_rate  # should be 2048 Hz sample rate. Need mat though..should i save this somehow?
        sig_data_in_seconds = np.array(mat[channel_to_index[channel]])
        ax.plot(HG_ev1_evoke.times, HG_ev1_evoke.data[channel_to_index[channel], :])
         # Add the standard error shading
        ax.fill_between(HG_ev1_evoke.times, HG_ev1_evoke.data[channel_to_index[channel], :] - stderr, HG_ev1_evoke.data[channel_to_index[channel], :] + stderr, alpha=0.2)

        # Find the maximum y-value for the current channel
        max_y_value = np.max(HG_ev1_evoke.data[channel_to_index[channel], :])

        # Overlay significance as a horizontal line at the max y-value
        significant_points = np.where(sig_data_in_seconds == 1)[0]
        for point in significant_points:
            ax.hlines(y=max_y_value, xmin=time_in_seconds[point]-1, xmax=time_in_seconds[point] + 0.005 - 1, color='red', linewidth=1) # subtract 1 cuz the sig time is from 0 to 2.5, while the high gamma time is from -1 to 1.5

        ax.set_title(channel)

    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    return fig

# Iterate over all channels in chunks of 60 and plot them
for i in range(0, len(sig_chans), 60):
    channels_subset = channels[i:i+60]
    fig = plot_channels_on_grid_time_perm_cluster_raw(channels_subset)
    combined_plot_path = os.path.join(save_dir, f'{sub}_raw_{output_name}_combinedChannelTracesAndTimePermClusterSignificance_Page_{i//36 + 1}.png')
    fig.savefig(combined_plot_path)
    plt.close(fig)
